In [1]:
import os
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.interpolate import interp1d
import plotly.express as px
import plotly.graph_objects as go
from openai import OpenAI

In [3]:
def load_rr_intervals(file_path):
    with open(file_path, 'r') as file:
        rr_intervals = [float(line.strip()) for line in file if line.strip() and line.strip().replace('.', '', 1).isdigit()]
    return rr_intervals

In [4]:
def calculate_heart_rate(rr_intervals):
    time_beats = np.cumsum(rr_intervals)/1000
    heart_rates = [60000 / i for i in rr_intervals]
    time_seconds = np.arange(0, time_beats[-1], step=1)

    # Interpolate heart rate values at each second
    heart_rate_interpolator = interp1d(time_beats, heart_rates, bounds_error=False, fill_value="extrapolate")
    heart_rate_seconds = heart_rate_interpolator(time_seconds)
    return heart_rate_seconds  # HR in beats per minute

In [5]:
def calculate_rmssd(rr_intervals):
    rr_intervals = np.array(rr_intervals, dtype=np.float64)
    # Remove nan values
    rr_intervals = rr_intervals[~np.isnan(rr_intervals)]
    if len(rr_intervals) < 2:
        return np.nan
    rr_diff = np.diff(rr_intervals)
    rr_sqdiff = rr_diff ** 2
    rmssd = np.sqrt(np.nanmean(rr_sqdiff))
    return rmssd

In [6]:
def calculate_sdnn(rr_intervals):
    return np.std(rr_intervals)

In [7]:
def calculate_pnn50(rr_intervals):
    differences = np.abs(np.diff(rr_intervals))
    count_nn50 = np.sum(differences > 50)  # Number of successive intervals that differ by more than 50 ms
    pnn50 = (count_nn50 / len(differences)) * 100
    return pnn50

In [8]:
def calculate_sd1_sd2(rr_intervals):
    sd1 = np.std(np.diff(rr_intervals) / np.sqrt(2))
    sd2 = np.sqrt(2 * np.var(rr_intervals) - sd1**2)
    return sd1, sd2

In [9]:
def rolling_rmssd(rr_intervals, window_size=60, step_size=5):
    intervals = np.array(rr_intervals)
    # Convert RR intervals to R-peak times in seconds
    r_peaks_times = np.cumsum(intervals) / 1000.0  # Convert to seconds
    total_time = r_peaks_times[-1]
    if total_time < window_size:
        window_size = total_time
    # Generate time points for rolling calculation
    t0_list = np.arange(0, total_time - window_size + step_size, step_size)
    times = []
    rmssd_values = []
    for t0 in t0_list:
        t1 = t0 + window_size
        # Find indices of RR intervals within the window
        idx = np.where((r_peaks_times >= t0) & (r_peaks_times < t1))[0]
        if len(idx) > 1:
            # Extract the RR intervals in the window
            rr_win = intervals[idx]
            rmssd = calculate_rmssd(rr_win)
            rmssd_values.append(rmssd)
            times.append(t0 + window_size / 2.0)
        else:
            # Not enough data to compute RMSSD
            rmssd_values.append(np.nan)
            times.append(t0 + window_size / 2.0)
    return times, rmssd_values

In [10]:
def rolling_rmssd_summary(rmssd_values):
    rolling_rmssd_summary = {
        'mean': np.mean(rmssd_values),
        'min': np.nanpercentile(rmssd_values, 5),
        '25%': np.nanpercentile(rmssd_values, 25),
        '50%': np.nanpercentile(rmssd_values, 50),
        '75%': np.nanpercentile(rmssd_values, 75),
        'max': np.nanpercentile(rmssd_values, 95),
    }
    return rolling_rmssd_summary

In [35]:
def plot_rolling_hrv(rr_intervals, window_size=300, step_size=5):
    times, rmssd_values = rolling_rmssd(rr_intervals, window_size=300, step_size=5)
    rmssd_df = pd.DataFrame({'Time (s)': times, 'RMSSD (ms)': rmssd_values})    
    def seconds_to_english_time(seconds):
        if seconds < 60:
            # Less than 1 minute
            seconds = seconds
            parts = []
            if seconds > 0:
                parts.append(f"{seconds} Second")
            return ' '.join(parts)
        elif seconds < 3600:
            # Less than 1 hour
            minutes = seconds // 60
            seconds = seconds % 60
            parts = []
            if minutes > 0:
                parts.append(f"{minutes} Minute")
            if seconds > 0:
                parts.append(f"{seconds} Second")
            return ' '.join(parts)
        else:
            # 1 hour or more
            hours = seconds // 3600
            remainder = seconds % 3600
            minutes = remainder // 60
            parts = []
            if hours > 0:
                parts.append(f"{hours} Hour")
            if minutes > 0:
                parts.append(f"{minutes} Minute")
            return ' '.join(parts)
    times, rmssd_values = rolling_rmssd(rr_intervals, window_size=300, step_size=5)
    rmssd_df = pd.DataFrame({'Time (s)': times, 'RMSSD (ms)': rmssd_values})    
    fig = px.line(
        rmssd_df,
        x='Time (s)',
        y='RMSSD (ms)',
        title= f'Rolling RMSSD Every {seconds_to_english_time(step_size)}s Over {seconds_to_english_time(window_size)} Windows',
        labels={'Time (s)': 'Time (s)', 'RMSSD (ms)': 'RMSSD (ms)'},
        markers=True
    )
    fig.update_layout(
        xaxis=dict(title='Time (s)'),
        yaxis=dict(title='RMSSD (ms)'),
        hovermode='x unified',
        template='plotly_white'
    )
    fig.show()

In [12]:
def calculate_lf_hf_ratio(rr_intervals, fs):
    time = np.cumsum(rr_intervals) / 1000  # Convert RR intervals to seconds
    rr_interpolated = np.interp(np.arange(time[0], time[-1], 1/fs), time, rr_intervals)

    # Calculate power spectral density using Welch's method
    freqs, psd = welch(rr_interpolated, fs=fs, nperseg=len(rr_interpolated) // 2)
    
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)
    
    lf_power = np.trapz(psd[(freqs >= lf_band[0]) & (freqs < lf_band[1])], freqs[(freqs >= lf_band[0]) & (freqs < lf_band[1])])
    hf_power = np.trapz(psd[(freqs >= hf_band[0]) & (freqs < hf_band[1])], freqs[(freqs >= hf_band[0]) & (freqs < hf_band[1])])
    
    return lf_power, hf_power, lf_power / hf_power

In [13]:
def hr_chart(hr_data, age):
    max_hr = 220 - age
    zones = {
        'Below Zone 1': (0, 0.5 * max_hr),
        'Zone 1': (0.5 * max_hr, 0.6 * max_hr),
        'Zone 2': (0.6 * max_hr, 0.7 * max_hr),
        'Zone 3': (0.7 * max_hr, 0.8 * max_hr),
        'Zone 4': (0.8 * max_hr, 0.9 * max_hr),
        'Zone 5': (0.9 * max_hr, max_hr),
        'Above Zone 5': (max_hr, 220)
    }
    zone_colors = {
        'Below Zone 1': '#00B050',
        'Zone 1': '#FFE699',
        'Zone 2': '#F4B183',
        'Zone 3': '#C55A11',
        'Zone 4': '#F36A53',
        'Zone 5': '#FF0000',
        'Above Zone 5': '#C00000'
    }
    fig = px.line(hr_data, x='Time', y='heart_rate', title='Heart Rate Over Time', labels={'Time': 'Time', 'heart_rate': 'Heart Rate'})
    fig.update_traces(line=dict(color='blue'))
    fig.update_xaxes(tickformat="%H:%M:%S", linecolor='black', mirror=True)
    fig.update_yaxes(range=[0, 220], linecolor='black', mirror=True)
    shapes = []
    for zone_name, (low, high) in zones.items():
        shapes.append(go.layout.Shape(
            type="rect",
            xref="paper", yref="y",
            x0=0, x1=1, y0=low, y1=high,
            fillcolor=zone_colors[zone_name], opacity=0.3,
            layer="below", line=dict(width=0)
        ))
    fig.update_layout(
        shapes=shapes,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40),
        showlegend=False
    )
    return fig

In [14]:
def analyze_rr_session(rr_intervals, fs, lf_hf_thresh, acute_lf_hf_thresh, start_time, end_time, window_size=60, step_size=5):
    avg_hr = np.mean(calculate_heart_rate(rr_intervals))
    hr_range = f"{np.round(np.min(calculate_heart_rate(rr_intervals)), 0)} - {np.round(np.max(calculate_heart_rate(rr_intervals)), 0)}"
    sd1, sd2 = calculate_sd1_sd2(rr_intervals)
    rmssd = calculate_rmssd(rr_intervals)
    sdnn = calculate_sdnn(rr_intervals)
    pnn50 = calculate_pnn50(rr_intervals)
    lf_power, hf_power, lf_hf_ratio = calculate_lf_hf_ratio(rr_intervals, fs)
    rmssd_summary = rolling_rmssd_summary(rolling_rmssd(rr_intervals, window_size, step_size)[1])
    HRV_mean = rmssd_summary['mean']
    HRV_median = rmssd_summary['50%']
    HRV_range = f"{np.round(rmssd_summary['min'], 0)} - {np.round(rmssd_summary['max'], 0)}"
    
    # Determine sympathetic or parasympathetic state
    state = "Parasympathetic"
    if lf_hf_ratio > lf_hf_thresh:
        state = "Sympathetic"
    if lf_hf_ratio > acute_lf_hf_thresh:
        state = "Acute Sympathetic"
    results = {
        "Time": f"{start_time} - {end_time}",
        "Avg HR (bpm)": np.round(avg_hr, 0),
        "HR Range": hr_range,
        "HRV": np.round(rmssd, 0),
        "SDNN": np.round(sdnn, 0),
        "HRV_mean": np.round(HRV_mean, 0),         
        "HRV_median": np.round(HRV_median, 0), 
        "HRV_range": HRV_range,         
        "pNN50 (%)": np.round(pnn50, 1),
        "SD1": np.round(sd1, 0),
        "SD2": np.round(sd2, 0),
        "LF Power": np.round(lf_power, 0),
        "HF Power": np.round(hf_power, 0),
        "LF/HF Ratio": np.round(lf_hf_ratio, 1),
        "State": state
    }
    return results

In [15]:
def analyze_rr_chunks(rr_intervals, fs, lf_hf_thresh, acute_lf_hf_thresh, chunk_duration=None):
    results = []
    if chunk_duration:
        chunk_duration_secs = chunk_duration * 60  # Convert to seconds
        chunks = [(i, i+chunk_duration_secs) for i in range(0, int(sum(rr_intervals)/1000), chunk_duration_secs)]
        rri_time_df = pd.DataFrame({'RRI': rr_intervals, 'Time': pd.Series(rr_intervals).cumsum() / 1000})
        rri_time_df['time_hhmmss'] = pd.to_timedelta(rri_time_df['Time'], unit='s').apply(lambda x: str(x).split('.')[0].split(" ")[-1])
        rr_intervals_chunks = []
        for times in chunks:
            rr_intervals_chunks.append(rri_time_df[(rri_time_df.Time > times[0]) & (rri_time_df.Time <= times[1])].RRI.values.tolist())
        for i, chunk in enumerate(rr_intervals_chunks):
            results.append(analyze_rr_session(chunk, fs, lf_hf_thresh, acute_lf_hf_thresh, start_time=i * chunk_duration, end_time=int(np.ceil(min((i + 1) * chunk_duration, sum(rr_intervals)/60000)))))
    return results

In [16]:
def plot_hr_with_sympathetic(rr_intervals, results=None, chunks=False):
    hr = calculate_heart_rate(rr_intervals)
    time = np.arange(len(hr)) / 60  # Convert to minutes

    # Create a DataFrame to store heart rate and time
    data = pd.DataFrame({
        'Time (minutes)': time,
        'Heart Rate (bpm)': hr
    })

    # Create the base line chart for heart rate
    fig = px.line(data, x='Time (minutes)', y='Heart Rate (bpm)', title="Heart Rate with Sympathetic/Parasympathetic State")

    if chunks and results:
        # Overlay shaded regions for sympathetic/parasympathetic states
        for result in results:
            start_time = int(result["Time"].split(" - ")[0])
            end_time = int(result["Time"].split(" - ")[1])
            
            if result["State"] == "Acute Sympathetic":
                color = 'red'
            elif result["State"] == "Sympathetic":
                color = 'orange'
            else:
                color = 'green'

            fig.add_shape(type="rect", x0=start_time, x1=end_time, y0=0, y1=220,
                          fillcolor=color, opacity=0.2, layer="below", line=dict(width=0))

    return fig

In [17]:
def openai_response(prompt, model = "gpt-4o"):
    client = OpenAI(api_key= openai_api_key.replace('"',''))
    response = client.chat.completions.create(
        model= model,
        messages=[ {"role": "user", "content": f"{prompt}"}]
        )
    return response.choices[0].message.content.strip()

In [41]:
def session_analysis(file_path, chunk_duration= None, fs=4, lf_hf_thresh=1.8, acute_lf_hf_thresh=3., start_time= 0, end_time= None, window_size=120, step_size=5):
    rr_intervals = load_rr_intervals(file_path)
    if not end_time:
        end_time= int(np.ceil(sum(rr_intervals)/60000))
    analysis_results = analyze_rr_session(rr_intervals, fs, lf_hf_thresh, acute_lf_hf_thresh, start_time, end_time, window_size, step_size)
    print(f"Full session analysis: \n{analysis_results}\n")
    plot_rolling_hrv(rr_intervals, window_size, step_size)
    if chunk_duration:
        chunk_analysis = analyze_rr_chunks(rr_intervals, fs, lf_hf_thresh, acute_lf_hf_thresh, chunk_duration=chunk_duration)
        print(f"Analysis of chunks: \n{pd.DataFrame(chunk_analysis)}\n")
        fig = plot_hr_with_sympathetic(rr_intervals, chunk_analysis)
        fig.show()
        for i, row in enumerate(chunk_analysis):
            lf = row['LF Power']
            hf = row['HF Power']
            lf_hf_ratio = np.round(lf/hf, 2)
            time = row['Time']
            HRV_mean = row["HRV_mean"]
            HRV_median = row["HRV_median"]
            HRV_range = row["HRV_range"]            
            prompt = f"""
            Here are the LF and HF and lf_hf_ratio from the Frequency Domain Analysis as well as the HRV_mean, HRV_median and HRV_range of the RR Intervals for a session: lf:{lf}, lf:{hf}, lf:{lf_hf_ratio}, lf:{HRV_mean}, lf:{HRV_median}, lf:{HRV_range}
            Based on this data, please provide a layman-friendly explanation of what this means and whether they're in a relaxed or stressed situation and the impact this has on decision making. I define a sympathetic state as one with a lf/hf ratio above 1.8 and an LF/HF ratio above 3 to be in acute sympathetic state. If they're in a sympathetic state, give a quick tip on how to bring themselves back into a parasympathetic state. Respond in one paragraph, avoiding technical jargon and calling out any of the metrics and their values explicitly.
            Do not explicitly call out the threshold values I provided. And address the reader as 'you'. Do not start with 'Based on the provided data' or similar
            """
            for i,j in row.items():
                print(f"{i}: {j}")
            print(f"Analysis: \n{openai_response(prompt)}\n")

In [19]:
file_path = 'HBM_Session_Data/2024-08-08 01-37-06.txt' 

In [42]:
session_analysis(file_path, chunk_duration= 45)

Full session analysis: 
{'Time': '0 - 271', 'Avg HR (bpm)': 58.0, 'HR Range': '25.0 - 156.0', 'HRV': 120.0, 'SDNN': 148.0, 'HRV_mean': 106.0, 'HRV_median': 80.0, 'HRV_range': '61.0 - 259.0', 'pNN50 (%)': 49.8, 'SD1': 85.0, 'SD2': 191.0, 'LF Power': 8787.0, 'HF Power': 3753.0, 'LF/HF Ratio': 2.3, 'State': 'Sympathetic'}

